## Пример построения  двухслойной нейронной сети на numpy

In [2]:
'''
Построение двухслойной нейронный сети для классификации цветков ириса
'''
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# sklearn здесь только, чтобы разделить выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split

### Шаг 1. Определение функций, которые понадобяться для обучения
# преобразование массива в бинарный вид результатов
def to_one_hot(Y):
    n_col = np.amax(Y) + 1
    binarized = np.zeros((len(Y), n_col))
    for i in range(len(Y)):
        binarized[i, Y[i]] = 1.
    return binarized

# преобразование массива в необходимый вид
def from_one_hot(Y):
    arr = np.zeros((len(Y), 1))

    for i in range(len(Y)):
        l = layer2[i]
        for j in range(len(l)):
            if(l[j] == 1):
                arr[i] = j+1
    return arr

# сигмоида и ее производная
def sigmoid(x):
    return 1/(1+np.exp(-x))
#    return x * (x>0)

def sigmoid_deriv(x):
    return sigmoid(x)*(1 - sigmoid(x))
#    return 1.0*(x>=0)

# нормализация массива
def normalize(X, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)

# вычисление точности предсказания по вероятности выбора и соответствующей ван-хот-метке этого же набора
def chck_acc(ar_prob, ar_lbl):
    pred = np.argmax(ar_prob,axis=1)
    pred = to_one_hot(pred) #.astype(np.int)
    kv,kh = ar_lbl.shape
    return 100* sum(sum(pred==ar_lbl))/(kv*kh)

def prediction(X_, w0,w1):
    layer0 = X_train
    layer1 = sigmoid(np.dot(X_,     w0))
    y_ = sigmoid(np.dot(layer1, w1))
    return y_

In [3]:
### Шаг 2. Подготовка тренировочных данных
# получения данных из csv файла. укажите здесь путь к файлу Iris.csv
#iris_data = pd.read_csv("lesson_source/Iris.csv")
iris_data = pd.read_csv("Iris.csv")

# print(iris_data.head()) # расскоментируйте, чтобы посмотреть структуру данных

# репрезентация данных в виде графиков
#g = sns.pairplot(iris_data.drop("Id", axis=1), hue="Species")
# plt.show() # расскоментируйте, чтобы посмотреть

# замена текстовых значений на цифровые
iris_data['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# формирование входных данных
columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
x = pd.DataFrame(iris_data, columns=columns)
x = normalize(x.as_matrix())

# формирование выходных данных(результатов)
columns = ['Species']
y = pd.DataFrame(iris_data, columns=columns)
y = y.as_matrix()
y = y.flatten()
y = to_one_hot(y)

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)


In [4]:
### Шаг 3. Обученние нейронной сети
# присваевание случайных весов

N1 = 5 # 1000 # 5
np.random.seed(1) # 11
w0 = 2*np.random.random((4, N1)) - 1 # для входного слоя   - 4 входа, 3 выхода
w1 = 2*np.random.random((N1, 3)) - 1 # для внутреннего слоя - 5 входов, 3 выхода

# скорость обучения (learning rate)
n = 0.12#  0.1# 0.12*0.95 -sigm # 0.008*0.9
acc_last = 0
# массив для ошибок, чтобы потом построить график
errors = []

# процесс обучения
#for i in range(100000):
for i in range(5000):

    # прямое распространение(feed forward)
    layer0 = X_train
    layer1 = sigmoid(np.dot(layer0, w0))
    layer2 = sigmoid(np.dot(layer1, w1))
    # обратное распространение(back propagation) с использованием градиентного спуска
    layer2_error = y_train - layer2
    layer2_delta = layer2_error * sigmoid_deriv(layer2)
    
    layer1_error = layer2_delta.dot(w1.T)
    layer1_delta = layer1_error * sigmoid_deriv(layer1)
    
    w1 += layer1.T.dot(layer2_delta) * n
    w0 += layer0.T.dot(layer1_delta) * n
    
    error = np.mean(np.abs(layer2_error))
#    error = np.mean(np.sqrt(layer2_error*layer2_error))

    errors.append(error)
    accuracy = (1 - error) * 100
#    accuracy = chck_acc(layer2, y_train)
    if i%10==0: 
        if acc_last >= accuracy: n = n*0.95
        acc_last = accuracy
    if i%100==0:
        print(i,n,accuracy)
        

### Шаг 4. Демонстрация полученных результатов
# черчение диаграммы точности в зависимости от обучения
#plt.plot(errors)
#plt.xlabel('Обучение')
#plt.ylabel('Ошибка')
#plt.show() # расскоментируйте, чтобы посмотреть 
        
print("Точность нейронной сети на тренировочном наборе " + str(round(accuracy,2)) + "%")
#accuracy = chck_acc(layer2, y_train)
#print("Точность нейронной сети на тренировочном наборе  " + str(round(accuracy,2)) + "%")

y_pred = prediction(X_train, w0,w1)
accuracy = chck_acc(y_pred, y_train)
print("Точность нейронной сети на тренировочном наборе  " + str(round(accuracy,2)) + "%")

y_pred = prediction(X_test, w0,w1)
accuracy = chck_acc(y_pred, y_test)
print("Точность нейронной сети на тестовом наборе  " + str(round(accuracy,2)) + "%")


0 0.12 47.49883774740075
100 0.12 80.28990926882247
200 0.12 82.96802090278537
300 0.12 85.81394659823492
400 0.12 88.77218618463749
500 0.12 91.16294745295829
600 0.12 92.3859123089577
700 0.12 93.35027108743017
800 0.12 94.21615873337389
900 0.12 94.95050260896825
1000 0.12 95.55094672512404
1100 0.12 96.0358034386732
1200 0.12 96.4283148724508
1300 0.12 96.74899582850355
1400 0.12 97.01397825528754
1500 0.12 97.23551812485462
1600 0.12 97.42285205482789
1700 0.12 97.58295650711352
1800 0.12 97.72113806945752
1900 0.12 97.84147149601857
2000 0.12 97.94711879686993
2100 0.12 98.04056042021722
2200 0.12 98.12376303822185
2300 0.12 98.19830203731307
2400 0.12 98.26545167685121
2500 0.12 98.32625207931787
2600 0.12 98.38155950545742
2700 0.12 98.43208446830512
2800 0.12 98.47842091667994
2900 0.12 98.5210687966603
3000 0.12 98.56045165463695
3100 0.12 98.59693049169168
3200 0.12 98.6308147572723
3300 0.12 98.6623711400928
3400 0.12 98.69183064828782
3500 0.12 98.7193943501331
3600 0.12 9

In [ ]:
print(x.shape, y.shape)

## Практическое задание

<ol>
    <li>Попробуйте видоизменить параметры разобранной на уроке двухслойной нейронной сети таким образом, чтобы улучшить ее точность. Проведите анализ — что приводит к ухудшению точности нейронной сети? Что приводит к увеличению ее точности?</li>
</ol>

Параметрами задачи являются:
    скорость обучения (n), число эпох, метрика, функция активации, алгоритм минимизации (менять не буду), 
    критерий качества (accuracy), размер входного слоя (N1)

Для экспериментирования сделал несколько модификаций кода: 
    - для финальной величины качества аккуратность рассчитывал по классической формуле (TP+TN / TP+TN+FP+FN)
    - скорость обучения сделал переменной, уменьшающейся по мере приближения к оптимуму
    - зафиксировал исходный рандом: np.random.seed(1) 

Попытка сильного увеличения стартовой скорости обучения приводит к расходимости алгоритма спуска (появляются nan)  
Для функции активации sigmoid начальную скорость можно ставить n=0.12, для ReLu n=0.01
Функция активации sigmoid более подходящая для задачи, хотя ReLu вначале и сходится быстрее, зато потом останавливается,
    не доходя до минимума ошибки. Для ReLu достаточно делать 400 шагов, а для sigmoid нужны - 4000 шагов.
    Однако, обучение с большим числом эпох приводит на малоразмерном входном слое к переобучению (tren- 100%, test - 90%)

Размер входного слоя 5 оказался оптимальным при меньшем размере качество ответов ухудшается, 
при большем - скорость минимизации снижается, но итоговая точность (по заданному в программе алгоритму - вероятность) 
слегка повышается, однако же при всём - количество угаданных сэмплов остается тем же!!
    